## 0. 전반적인 Process
- 현황
    - 샘플 데이터를 활용한 Prompt Engineering 수행 중
- 과제
    - Postgre DB 연계 필요
    - BPI 연계 필요

In [ ]:
from IPython.display import Image, display
import base64

def mm(graph: str) -> None:
    """
    Mermaid 그래프를 이미지로 렌더링하여 표시합니다.

    이 함수는 Mermaid 문법으로 작성된 그래프 문자열을 받아
    base64로 인코딩한 후, Mermaid 온라인 렌더링 서비스를 통해
    이미지로 변환하여 Jupyter Notebook이나 유사한 환경에서 표시합니다.

    Args:
        graph (str): Mermaid 문법으로 작성된 그래프 문자열

    Returns:
        None: 이미지를 직접 표시하므로 반환값은 없습니다.

    Example:
        mm('''
        graph TD
        A[Start] --> B{Is it?}
        B -- Yes --> C[OK]
        B -- No --> D[End]
        ''')
    """
    # 그래프 문자열을 UTF-8로 인코딩
    graphbytes = graph.encode("utf-8")
    
    # 인코딩된 바이트를 base64로 변환
    base64_bytes = base64.b64encode(graphbytes)
    
    # base64 바이트를 ASCII 문자열로 디코딩
    base64_string = base64_bytes.decode("ascii")
    
    # Mermaid 온라인 렌더링 서비스 URL과 base64 문자열을 결합하여 이미지 표시
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
graph LR
    A[API Key 입력] --> B[PostGre DB 연계]
    B --> C[BPI 계산]
    C --> D[강약점 선택]
    D --> E[강약점 데이터전처리]
    E --> F[Prompt Engineering]
    F --> G[완료]
""")

## 1. 필수 라이브러리 임포트 및 환경 설정
- .env 파일에 '자신' 또는 '더존' api key 저장: 
    * OPENAI_API_KEY=sk-proj-.....

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Dict, Any, Literal
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain.cache import SQLiteCache
from operator import itemgetter
from tabulate import tabulate
from langchain.callbacks import get_openai_callback


# env 파일 읽기
load_dotenv()

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

## 2. Sample Data 생성
- 데이터 출력 방식은 협의 필요 (\w 조희성 과장님, 이도은 대리님)
- 현재 출력물
    - df_company_info : 회사명, 업종, 재무/회계 데이터
    - df_partner_info : df_company_info의 회사의 매입/매출 거래처 정보 등
- 향후 : Postgre DB와 연계

In [ ]:
import pandas as pd
import numpy as np

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

# 재현 가능성을 위해 난수 생성기의 시드를 설정합니다.
np.random.seed(42)

# 회사 및 업종 정보 생성
num_companies = 100  # 총 100개의 회사 생성
company_names = [f'Company_{i+1}' for i in range(num_companies)]  # 회사명 리스트 생성 (Company_1, Company_2, ..., Company_100)
industries = ['Manufacturing', 'Retail', 'Technology', 'Healthcare', 'Finance']  # 업종 리스트 정의

# 각 회사에 대한 고정된 업종 리스트 생성
company_industries = [np.random.choice(industries) for _ in range(num_companies)]  

# 날짜 생성 (2020-01 ~ ACCESS_TIME 기준 한 달 전까지)
dates = pd.date_range(start="2020-01-01", end=access_datetime - pd.DateOffset(months=1), freq='MS').strftime("%Y-%m").tolist()

# 상위 매출처 및 매입처 정보 생성 함수 정의
sales_partner_pool = list(set(f'SalesPartner_{i+1}' for i in range(200)))  # 매출처 Pool 생성 (200개)
purchase_partner_pool = list(set(f'PurchasePartner_{i+1}' for i in range(200)))  # 매입처 Pool 생성 (200개)

# 각 회사별 기본 인원수를 설정합니다.
company_base_employees = {company: np.random.randint(100, 300) for company in company_names}  # 기업별로 100명 ~ 300명 사이의 기본 인원수 설정

def generate_partner_data():
    """
    상위 매출처 및 매입처 정보를 생성하는 함수입니다.
    
    Returns:
        dict: 생성된 매출처 및 매입처 정보 딕셔너리 리스트
    """
    # 상위 5개 매출처 및 매입처 무작위 선택
    sales_companies = list(np.random.choice(sales_partner_pool, 5, replace=False))
    sales_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))
    sales_concentration = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))  # 매출처 집중도

    purchase_companies = list(np.random.choice(purchase_partner_pool, 5, replace=False))
    purchase_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))
    purchase_concentration = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))  # 매입처 집중도

    # 매출처 및 매입처 정보 딕셔너리 리스트로 생성
    sales_partners = [
        {
            "매출처_회사명": sales_companies[i],
            "매출처_신용등급": sales_grades[i],
            "매출처_집중도": sales_concentration[i],
            "월별_거래금액": round(np.random.uniform(500, 5000), 0),  # 월별 거래금액 (500만원 ~ 5000만원)
            "월별_회수기일": round(np.random.uniform(30, 90), 0),  # 월별 회수기일 (30일 ~ 90일)
            "월별_회수잔액(예정)": round(np.random.uniform(100, 1000), 0)  # 월별 회수잔액 (100만원 ~ 1000만원)
        }
        for i in range(5)
    ]

    purchase_partners = [
        {
            "매입처_회사명": purchase_companies[i],
            "매입처_신용등급": purchase_grades[i],
            "매입처_집중도": purchase_concentration[i],
            "월별_거래금액": round(np.random.uniform(500, 5000), 0),  # 월별 거래금액 (500만원 ~ 5000만원)
            "월별_회수기일": round(np.random.uniform(30, 90), 0),  # 월별 회수기일 (30일 ~ 90일)
            "월별_회수잔액(예정)": round(np.random.uniform(100, 1000), 0)  # 월별 회수잔액 (100만원 ~ 1000만원)
        }
        for i in range(5)
    ]

    return sales_partners, purchase_partners

# 기본 정보 데이터 생성 함수 정의
def generate_basic_data(company, industry, date):
    """
    각 회사의 특정 날짜에 대한 기본 재무 데이터를 생성하는 함수입니다.
    
    Args:
        company (str): 회사명
        industry (str): 업종명
        date (str): 날짜 (YYYY-MM 형식)
    
    Returns:
        dict: 생성된 회사의 기본 재무 정보 및 상위 매출처, 매입처 정보
    """
    base_asset_value = 30000  # 기본 총자산 값 설정 (3억원)
    total_assets = round(base_asset_value * (1 + np.random.normal(0, 0.05)), 0)  # 기본 총자산 값에서 5% 변동 (정규분포 이용, 소수점 없이)
    revenue = round(np.random.uniform(1000, 10000), 0)  # 매출액은 1천만원 ~ 1억원 사이의 값으로 설정 (소수점 없이)
    operating_profit = round(revenue * np.random.uniform(-0.2, 0.2), 0)  # 영업이익은 매출액의 -20%~20% 범위로 설정 (소수점 없이)
    net_income = round(operating_profit * np.random.uniform(-0.5, 0.8), 0)  # 당기순이익은 영업이익의 -50%~80% 범위로 설정 (소수점 없이)
    short_term_loans = round(np.random.uniform(500, 5000), 0)  # 단기차입금은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    long_term_loans = round(np.random.uniform(1000, 10000), 0)  # 장기차입금은 1000만원 ~ 1억원 범위로 설정 (소수점 없이)
    total_loans = short_term_loans + long_term_loans  # 총 차입금 계산
    loan_to_sales = round((total_loans / revenue) * 100, 2)  # 매출대비차입금 비율을 백분율로 계산
    working_capital_turnover = round(np.random.uniform(1, 5), 2)  # 운전자금회전율은 1~5회 사이의 값으로 설정
    operating_cash_flow = round(np.random.uniform(500, 5000), 0)  # 영업활동현금흐름은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    net_cash_flow = round(operating_cash_flow - total_loans, 0)  # 순현금흐름 = 영업활동현금흐름 - 총 차입금 (소수점 없이)
    ar_balance = round(np.random.uniform(1000, 5000), 0)  # 매출채권 규모 (1000만원 ~ 5000만원)
    ap_balance = round(np.random.uniform(800, 4000), 0)  # 매입채무 규모 (800만원 ~ 4000만원)
    inventory = round(np.random.uniform(2000, 10000), 0)  # 재고자산 (2000만원 ~ 1억원)

    # 상위 매출처 및 매입처 정보 생성
    sales_partners, purchase_partners = generate_partner_data()

    # 기본 인원수를 기반으로 ±10% 내외의 변동을 줍니다.
    base_employees = company_base_employees[company]
    employees = round(base_employees * (1 + np.random.uniform(-0.1, 0.1)))  # ±10% 내외 변동

    return {
        '기업명': company,
        '업종': industry,
        '날짜': date,
        '매출액증가율': round(np.random.uniform(5, 15), 2),  # 매출액증가율 (5% ~ 15% 사이)
        '총자산증가율': round(np.random.uniform(3, 10), 2),  # 총자산증가율 (3% ~ 10% 사이)
        '총자산': int(total_assets),  # 총자산 (정수형)
        '매출액': int(revenue),  # 매출액 (정수형)
        '영업이익': int(operating_profit),  # 영업이익 (정수형)
        '영업이익률': round(operating_profit / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-20, 20), 2),  # 영업이익률 (백분율)
        '당기순이익': int(net_income),  # 당기순이익 (정수형)
        '당기순이익률': round(net_income / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-30, 30), 2),  # 당기순이익률 (백분율)
        '단기차입금': int(short_term_loans),  # 단기차입금 (정수형)
        '장기차입금': int(long_term_loans),  # 장기차입금 (정수형)
        '매출대비차입금': loan_to_sales,  # 매출대비차입금 비율 (백분율)
        '운전자금회전율': working_capital_turnover,  # 운전자금회전율
        '인원수': employees,  # 인원수 (±10% 변동)
        '월평균급여액': round(np.random.uniform(200, 500), 1),  # 월평균 급여액 (200 ~ 500만원)
        '월매출창출액': round(np.random.uniform(5000, 20000), 1),  # 월 매출 창출액 (5000 ~ 20000만원)
        '영업활동현금흐름/매출액': round(np.random.uniform(0.05, 0.2), 2),  # 영업활동현금흐름/매출액 비율 (5% ~ 20%)
        '영업활동현금흐름': int(operating_cash_flow),  # 영업활동현금흐름 (정수형)
        '순현금흐름': int(net_cash_flow),  # 순현금흐름 (정수형)
        '매출채권': int(ar_balance),  # 매출채권 (정수형)
        '매입채무': int(ap_balance),  # 매입채무 (정수형)
        '재고자산': int(inventory),  # 재고자산 (정수형)
        '상위_매출처': sales_partners,  # 상위 매출처 정보
        '상위_매입처': purchase_partners  # 상위 매입처 정보
    }

# 기본 정보 데이터 생성
# 각 회사와 날짜별로 기본 재무 데이터를 생성하여 리스트로 저장
basic_data = [
    generate_basic_data(company, industry, date)
    for company, industry in zip(company_names, company_industries)
    for date in dates
]

# 기본 정보 데이터프레임 생성
df_company_info = pd.DataFrame(basic_data)  # 회사 기본 정보로 데이터프레임 생성

# 데이터프레임 샘플 출력
print(df_company_info[['기업명', '날짜', '상위_매출처', '상위_매입처']].head())


- Company_1에 대한 샘플 데이터

In [ ]:
from pprint import pprint

print('#'*10, 'Company_1에 대한 정보', '#'*10)
pprint(df_company_info[df_company_info['기업명']== 'Company_1'])

df_company_info.to_csv("sample.csv", encoding='utf-8-sig', index=False)

## 3. 데이터전처리
- 현황 : 성장성, 수익성 구현
- 전처리 함수
    - preprocess_growth_data : '총자산', '매출액' 등 성장성 관련 Data를 Dict 형태로 변경
    - preprocess_profitability_data : '영업이익', '당기순이익' 등 수익성 관련 Data를 Dict 형태로 변경

### 성장성 (Growth) 전처리

In [67]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any, Optional
import logging
from pprint import pprint

# 재현성을 위한 Random seed 설정
np.random.seed(42)

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

class DataValidationError(Exception):
    """데이터 검증 관련 커스텀 예외"""
    pass

class GrowthMetricsCalculator:
    """성장성 지표 계산을 위한 메인 클래스"""
    
    # 클래스 상수 정의
    REQUIRED_COLUMNS = ['기업명', '날짜', '업종', '총자산', '매출액']
    ASSET_COL = '총자산'
    REVENUE_COL = '매출액'
    COMPANY_COL = '기업명'
    DATE_COL = '날짜'
    INDUSTRY_COL = '업종'
    
    def __init__(self, df_company_info: pd.DataFrame, target_company_name: str, n_years: int = 3):
        """
        Args:
            df_company_info: 전체 회사 재무 데이터
            target_company_name: 대상 회사명
            n_years: 분석할 과거 연도 수 (기본값: 3)
        """
        self._validate_input_data(df_company_info, target_company_name)
        self.df = df_company_info[self.REQUIRED_COLUMNS].copy()
        self.target_company_name = target_company_name
        self.n_years = n_years
        self.growth_data = {}

    def _validate_input_data(self, df: pd.DataFrame, company_name: str) -> None:
        """입력 데이터 유효성 검증"""
        try:
            if not all(col in df.columns for col in self.REQUIRED_COLUMNS):
                missing_cols = set(self.REQUIRED_COLUMNS) - set(df.columns)
                raise DataValidationError(f"필수 컬럼 누락: {missing_cols}")
            
            if company_name not in df[self.COMPANY_COL].unique():
                raise DataValidationError(f"회사를 찾을 수 없음: {company_name}")
            
            if df.empty:
                raise DataValidationError("빈 데이터프레임")
                
            numeric_cols = [self.ASSET_COL, self.REVENUE_COL]
            for col in numeric_cols:
                if not pd.to_numeric(df[col], errors='coerce').notnull().all():
                    raise DataValidationError(f"숫자가 아닌 값이 포함됨: {col}")
                    
            logger.info("데이터 검증 완료")
        except Exception as e:
            logger.error(f"데이터 검증 실패: {str(e)}")
            raise

    def _prepare_data(self) -> None:
        """기초 데이터 준비"""
        try:
            self.df['date'] = pd.to_datetime(self.df[self.DATE_COL])
            self.df['year'] = self.df['date'].dt.year
            self.df['month'] = self.df['date'].dt.month
            
            self._handle_missing_values()
            
            self.df_company = self.df[self.df[self.COMPANY_COL] == self.target_company_name].copy()
            self.df_company.reset_index(drop=True, inplace=True)
            
            if self.df_company.empty:
                raise DataValidationError(f"회사 데이터가 없음: {self.target_company_name}")
            
            self.target_industry = self.df_company[self.INDUSTRY_COL].iloc[0]
            self.df_industry = self.df[self.df[self.INDUSTRY_COL] == self.target_industry].copy()
            self.df_industry.reset_index(drop=True, inplace=True)
            
            # 접근 가능한 최신 날짜 설정
            self.latest_date = access_datetime - pd.DateOffset(months=1)
            self.latest_year = self.latest_date.year
            self.latest_month = self.latest_date.month
            
            logger.info(f"데이터 준비 완료 - 기준일자: {self.latest_date.strftime('%Y-%m-%d')}")
        except Exception as e:
            logger.error(f"데이터 준비 실패: {str(e)}")
            raise

    def _handle_missing_values(self) -> None:
        """결측치 처리"""
        numeric_cols = [self.ASSET_COL, self.REVENUE_COL]
        for col in numeric_cols:
            missing_count = self.df[col].isnull().sum()
            if missing_count > 0:
                logger.warning(f"{col} 컬럼 결측치 발견: {missing_count}개")
                self.df[col] = self.df[col].fillna(0)

    def _calculate_growth_rate(self, current: float, previous: float, threshold: float = 0.0001) -> float:
        """성장률 계산"""
        try:
            if abs(previous) < threshold:
                logger.warning(f"이전 값이 임계값보다 작음 (previous: {previous})")
                return 0
            growth_rate = round(((current - previous) / previous * 100), 2)
            logger.debug(f"성장률 계산: current={current}, previous={previous}, rate={growth_rate}")
            return growth_rate
        except Exception as e:
            logger.warning(f"성장률 계산 실패: {str(e)}")
            return 0

    def _process_historical_data(self) -> None:
        """과거 3년 데이터 처리"""
        try:
            for year_offset in range(1, 4):
                target_year = self.latest_year - year_offset
                year_key = f"{target_year}년"
                
                current_year_company = self.df_company[self.df_company['year'] == target_year]
                prev_year_company = self.df_company[self.df_company['year'] == (target_year - 1)]
                
                current_year_industry = self.df_industry[self.df_industry['year'] == target_year]
                prev_year_industry = self.df_industry[self.df_industry['year'] == (target_year - 1)]
                
                company_metrics = self._calculate_metrics(
                    current_year_company, prev_year_company,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=False
                )
                
                industry_metrics = self._calculate_metrics(
                    current_year_industry, prev_year_industry,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=True
                )
                
                self._update_growth_data(year_key, company_metrics, industry_metrics)
                logger.info(f"{year_key} 데이터 처리 완료")
        except Exception as e:
            logger.error(f"과거 데이터 처리 실패: {str(e)}")
            raise

    def _process_estimate_data(self) -> None:
        """연말 예상 데이터 처리"""
        try:
            estimate_key = f"{self.latest_year}년(E)"
            
            current_mask = (self.df_company['year'] == self.latest_year) & \
                          (self.df_company['month'] <= self.latest_month)
            current_company = self.df_company[current_mask].copy()
            current_company.reset_index(drop=True, inplace=True)
            
            industry_mask = (self.df_industry['year'] == self.latest_year) & \
                          (self.df_industry['month'] <= self.latest_month)
            current_industry = self.df_industry[industry_mask].copy()
            current_industry.reset_index(drop=True, inplace=True)
            
            # print('COMPANY')
            company_metrics = self._calculate_annualized_metrics(current_company, is_industry=False)
            # print('INDUSTRY')
            industry_metrics = self._calculate_annualized_metrics(current_industry, is_industry=True)
            
            self._update_growth_data(estimate_key, company_metrics, industry_metrics)
            logger.info("연말 예상 데이터 처리 완료")
        except Exception as e:
            logger.error(f"연말 예상 데이터 처리 실패: {str(e)}")
            raise

    def _process_recent_data(self) -> None:
        """최근 12개월 데이터 처리"""
        try:
            # latest_date부터 거슬러 올라가며 12개월간의 데이터
            past_12_months = pd.date_range(end=self.latest_date, periods=12, freq='M')
            cumulative_data = {'current': {'revenue': 0}, 'previous': {'revenue': 0}}

            for idx, date in enumerate(past_12_months):
                month_str = date.strftime('%Y-%m')
                # latest_date부터 시작하여 해당 월의 데이터 선택
                current_month = self.df_company[
                    self.df_company['date'].dt.strftime('%Y-%m') == month_str
                ]
                # 전년도 동일 월의 데이터 선택
                prev_year_month = (date - pd.DateOffset(years=1)).strftime('%Y-%m')
                prev_month = self.df_company[
                    self.df_company['date'].dt.strftime('%Y-%m') == prev_year_month
                ]

                # 월별 데이터 업데이트
                self._update_monthly_data(month_str, current_month, prev_month,
                                        cumulative_data, idx == 0)

            logger.info("최근 12개월 데이터 처리 완료")
        except Exception as e:
            logger.error(f"최근 데이터 처리 실패: {str(e)}")
            raise


    def _handle_missing_values(self) -> None:
        """결측치 처리"""
        numeric_cols = [self.ASSET_COL, self.REVENUE_COL]
        for col in numeric_cols:
            missing_count = self.df[col].isnull().sum()
            if missing_count > 0:
                logger.warning(f"{col} 컬럼 결측치 발견: {missing_count}개")
                self.df[col] = self.df[col].fillna(0)

    def _calculate_growth_rate(self, current: float, previous: float, threshold: float = 0.0001) -> float:
        """성장률 계산"""
        try:
            if abs(previous) < threshold:
                logger.warning(f"이전 값이 임계값보다 작음 (previous: {previous})")
                return 0
            growth_rate = round(((current - previous) / previous * 100), 2)

            logger.debug(f"성장률 계산: current={current}, previous={previous}, rate={growth_rate}")
            return growth_rate
        except Exception as e:
            logger.warning(f"성장률 계산 실패: {str(e)}")
            return 0

    def _process_historical_data(self) -> None:
        """과거 3년 데이터 처리"""
        try:
            for year_offset in range(1, 4):
                target_year = self.latest_year - year_offset
                year_key = f"{target_year}년"
                
                current_year_company = self.df_company[self.df_company['year'] == target_year]
                prev_year_company = self.df_company[self.df_company['year'] == (target_year - 1)]
                
                current_year_industry = self.df_industry[self.df_industry['year'] == target_year]
                prev_year_industry = self.df_industry[self.df_industry['year'] == (target_year - 1)]
                
                company_metrics = self._calculate_metrics(
                    current_year_company, prev_year_company,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=False
                )
                
                industry_metrics = self._calculate_metrics(
                    current_year_industry, prev_year_industry,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=True
                )
                
                self._update_growth_data(year_key, company_metrics, industry_metrics)
                logger.info(f"{year_key} 데이터 처리 완료")
        except Exception as e:
            logger.error(f"과거 데이터 처리 실패: {str(e)}")
            raise

    def _process_recent_data(self) -> None:
        """최근 12개월 데이터 처리"""
        try:
            # latest_date까지 포함하여 12개월 기간 생성
            end_date = self.latest_date
            start_date = end_date - pd.DateOffset(months=11)
            
            # 특정 start_date와 end_date 범위에서 월별 데이터 생성 (month-end가 아닌 날짜 포함)
            past_12_months = pd.date_range(start=start_date, end=end_date, freq='MS')
            
            # 누적 데이터 초기화
            cumulative_data = {'current': {'revenue': 0}, 'previous': {'revenue': 0}}

            # 최신 날짜부터 12개월 동안 반복하여 데이터를 처리합니다.
            for idx, date in enumerate(past_12_months):
                year = date.year
                month = date.month

                # 현재 연도와 월 데이터를 필터링합니다.
                current_month = self.df_company[
                    (self.df_company['year'] == year) & 
                    (self.df_company['month'] == month)
                ]

                # 전년도 동일 월의 데이터 선택
                prev_year = year - 1
                prev_month = self.df_company[
                    (self.df_company['year'] == prev_year) & 
                    (self.df_company['month'] == month)
                ]

                # 월별 데이터 업데이트
                month_str = f"{year}-{month:02d}"
                self._update_monthly_data(month_str, current_month, prev_month,
                                        cumulative_data, idx == 0)

            logger.info("최근 12개월 데이터 처리 완료")
        except Exception as e:
            logger.error(f"최근 데이터 처리 실패: {str(e)}")
            raise

    def _calculate_metrics(self, current_data: pd.DataFrame, prev_data: pd.DataFrame,
                        current_period: str, prev_period: str, is_industry: bool = False) -> Dict[str, float]:
        """성장성 지표 계산"""
        try:
            if current_data.empty:
                logger.warning(f"현재 데이터 없음: {current_period}")
                return {'total_assets': 0, 'revenue': 0, 'asset_growth_rate': 0, 'revenue_growth_rate': 0}
            
            if is_industry:
                
                # 업종 평균 계산 - 총자산은 12월 말 데이터 기준 / 매출액은 연간 합계
                current_assets = current_data.groupby('year')[self.ASSET_COL].last().values[0]

                current_revenue = current_data.groupby('year')[self.REVENUE_COL].sum().iloc[-1]
                # print(current_revenue)
                prev_assets = 0
                if not prev_data.empty:
                    prev_assets = prev_data.groupby('year')[self.ASSET_COL].last().values[0]

                prev_revenue = prev_data.groupby('year')[self.REVENUE_COL].sum().iloc[-1] if not prev_data.empty else 0
                # print(prev_revenue)
                # print(self._calculate_growth_rate(current_revenue, prev_revenue))
            else:
                # 개별 기업 계산 - 12월 말 데이터 사용
                current_assets = current_data[self.ASSET_COL].iloc[-1]
                current_revenue = current_data[self.REVENUE_COL].sum()
                prev_assets = prev_data[self.ASSET_COL].iloc[-1] if not prev_data.empty else 0
                prev_revenue = prev_data[self.REVENUE_COL].sum() if not prev_data.empty else 0
            
            metrics = {
                'total_assets': int(round(current_assets)),
                'revenue': int(round(current_revenue)),
                'asset_growth_rate': self._calculate_growth_rate(current_assets, prev_assets),
                'revenue_growth_rate': self._calculate_growth_rate(current_revenue, prev_revenue)
            }
            logger.debug(f"지표 계산 완료: {metrics}")
            return metrics
        except Exception as e:
            logger.error(f"지표 계산 실패: {str(e)}")
            return {'total_assets': 0, 'revenue': 0, 'asset_growth_rate': 0, 'revenue_growth_rate': 0}


    def _calculate_annualized_metrics(self, current_data: pd.DataFrame, is_industry: bool = False) -> Dict[str, float]:
        """연환산 지표 계산"""
        try:
            if current_data.empty:
                logger.warning("현재 데이터 없음")
                return {'total_assets': 0, 'revenue': 0, 'asset_growth_rate': 0, 'revenue_growth_rate': 0}
            
            monthly_avg_assets = current_data[self.ASSET_COL].mean()
            monthly_revenue = current_data[self.REVENUE_COL].sum()
            annualized_revenue = (monthly_revenue / self.latest_month) * 12
            
            # 전년도 데이터 가져오기
            prev_year_data = self._get_previous_year_data(current_data['year'].iloc[0], is_industry=is_industry)
            prev_assets = prev_year_data[self.ASSET_COL].iloc[-1] if not prev_year_data.empty else 0
            prev_revenue = prev_year_data[self.REVENUE_COL].sum() if not prev_year_data.empty else 0
            
            metrics = {
                'total_assets': int(round(monthly_avg_assets)),
                'revenue': int(round(annualized_revenue)),
                'asset_growth_rate': self._calculate_growth_rate(monthly_avg_assets, prev_assets),
                'revenue_growth_rate': self._calculate_growth_rate(annualized_revenue, prev_revenue)
            }
            logger.debug(f"연환산 지표 계산 완료: {metrics}")
            return metrics
        except Exception as e:
            logger.error(f"연환산 지표 계산 실패: {str(e)}")
            return {'total_assets': 0, 'revenue': 0, 'asset_growth_rate': 0, 'revenue_growth_rate': 0}


    def _update_monthly_data(self, month: str, current_data: pd.DataFrame,
                            prev_data: pd.DataFrame, cumulative_data: Dict[str, Dict[str, float]],
                            is_first_month: bool) -> None:
        """월별 데이터 업데이트"""
        try:
            if current_data.empty:
                self._set_default_monthly_values(month)
                return

            current_revenue = current_data[self.REVENUE_COL].iloc[0]
            prev_revenue = prev_data[self.REVENUE_COL].iloc[0] if not prev_data.empty else 0
            
            cumulative_data['current']['revenue'] += current_revenue
            if not prev_data.empty:
                cumulative_data['previous']['revenue'] += prev_revenue

            revenue_growth = self._calculate_growth_rate(current_revenue, prev_revenue)
            cumulative_growth = self._calculate_growth_rate(
                cumulative_data['current']['revenue'],
                cumulative_data['previous']['revenue']
            )

            if is_first_month:
                cumulative_growth = revenue_growth
            
            self.growth_data['recent_data'][month] = {
                '매출액': int(round(current_revenue)),
                '전년동월 매출액': int(round(prev_revenue)),
                '매출액증가율': revenue_growth,
                '누적 매출액증가율': cumulative_growth
            }
            logger.debug(f"{month} 월별 데이터 업데이트 완료")
        except Exception as e:
            logger.error(f"월별 데이터 업데이트 실패: {str(e)}")
            self._set_default_monthly_values(month)

    def _set_default_monthly_values(self, month: str) -> None:
        """월별 기본값 설정"""
        try:
            self.growth_data['recent_data'][month] = {
                '매출액': 0,
                '전년동월 매출액': 0,
                '매출액증가율': 0,
                '누적 매출액증가율': 0
            }
            logger.debug(f"{month} 월별 기본값 설정 완료")
        except Exception as e:
            logger.error(f"월별 기본값 설정 실패: {str(e)}")

    def _get_previous_year_data(self, current_year: int, is_industry: bool = False) -> pd.DataFrame:
        """전년도 데이터 조회 (업종 또는 개별 회사 데이터)"""
        try:
            if is_industry:
                # 업종 전체 데이터에서 전년도 데이터 선택
                prev_year_data = self.df_industry[self.df_industry['year'] == (current_year - 1)].copy()
            else:
                # 개별 회사 데이터에서 전년도 데이터 선택
                prev_year_data = self.df_company[self.df_company['year'] == (current_year - 1)].copy()
            
            prev_year_data.reset_index(drop=True, inplace=True)
            return prev_year_data
        except Exception as e:
            logger.error(f"전년도 데이터 조회 실패: {str(e)}")
            return pd.DataFrame()


    def _update_growth_data(self, key: str, company_metrics: Dict[str, float],
                            industry_metrics: Dict[str, float]) -> None:
        """성장성 데이터 업데이트"""
        try:
            self.growth_data['year_level_data'][key] = {
                '총자산': company_metrics['total_assets'],
                '매출액': company_metrics['revenue']
            }
            self.growth_data['year_rate_data'][key] = {
                '총자산증가율': company_metrics['asset_growth_rate'],
                '매출액증가율': company_metrics['revenue_growth_rate'],
                '업종_평균_총자산증가율': industry_metrics['asset_growth_rate'],
                '업종_평균_매출액증가율': industry_metrics['revenue_growth_rate']
            }
            logger.debug(f"{key} 성장성 데이터 업데이트 완료")
        except Exception as e:
            logger.error(f"성장성 데이터 업데이트 실패: {str(e)}")
            raise

    def calculate_growth_metrics(self) -> Dict[str, pd.DataFrame]:
        """성장성 지표 계산 실행"""
        try:
            self._prepare_data()
            
            self.growth_data = {
                'latest_year_month': self.latest_date.strftime('%Y-%m'),
                'year_level_data': {},
                'year_rate_data': {},
                'recent_data': {}
            }
            
            self._process_historical_data()
            self._process_estimate_data()
            self._process_recent_data()
            
            # 연간 데이터 재구성 (연도 순서 변경 및 전치)
            years = sorted(self.growth_data['year_level_data'].keys())  # 연도 오름차순 정렬
            
            annual_revenue = {
                '매출액': [int(self.growth_data['year_level_data'][year]['매출액']) for year in years],
                '매출액증가율': [self.growth_data['year_rate_data'][year]['매출액증가율'] for year in years],
                '업종평균 매출액증가율': [self.growth_data['year_rate_data'][year]['업종_평균_매출액증가율'] for year in years],
            }
            df_annual_revenue = pd.DataFrame(annual_revenue, index=years).T

            annual_assets = {
                '총자산': [self.growth_data['year_level_data'][year]['총자산'] for year in years],
                '총자산증가율': [self.growth_data['year_rate_data'][year]['총자산증가율'] for year in years],
                '업종평균 총자산증가율': [self.growth_data['year_rate_data'][year]['업종_평균_총자산증가율'] for year in years],
            }
            df_annual_assets = pd.DataFrame(annual_assets, index=years).T
            
            # 월별 데이터 재구성 (전치)
            months = sorted(self.growth_data['recent_data'].keys())  # 날짜 오름차순 정렬
            
            monthly_revenue = {
                '당월매출액': [self.growth_data['recent_data'][month]['매출액'] for month in months],
                '전년동월매출액': [self.growth_data['recent_data'][month]['전년동월 매출액'] for month in months]
            }
            df_monthly_revenue = pd.DataFrame(monthly_revenue, index=months).T
            
            monthly_growth = {
                '매출액증가율': [self.growth_data['recent_data'][month]['매출액증가율'] for month in months],
                '누적매출액증가율': [self.growth_data['recent_data'][month]['누적 매출액증가율'] for month in months],
            }
            df_monthly_growth = pd.DataFrame(monthly_growth, index=months).T
            
            return {
                'latest_year_month': self.latest_date.strftime('%Y-%m'),
                'annual_revenue': df_annual_revenue,
                'annual_assets': df_annual_assets,
                'monthly_revenue': df_monthly_revenue,
                'monthly_growth': df_monthly_growth
            }
            
        except Exception as e:
            logger.error(f"성장성 지표 계산 실패: {str(e)}")
            raise

def preprocess_growth_data(df_company_info: pd.DataFrame, target_company_name: str) -> Dict[str, Any]:
    """
    성장성 지표 데이터를 전처리하여 JSON 형식으로 변환
    
    Args:
        df_company_info: 전체 회사 재무 데이터
        target_company_name: 대상 회사명
        
    Returns:
        Dict[str, Any]: 전처리된 성장성 지표 데이터
    """
    try:
        calculator = GrowthMetricsCalculator(df_company_info, target_company_name)
        return calculator.calculate_growth_metrics()
    except Exception as e:
        logger.error(f"전처리 실패: {str(e)}")
        raise

growth_data = preprocess_growth_data(df_company_info, 'Company_1')
pprint(growth_data['latest_year_month'])

pprint(pd.DataFrame(growth_data['annual_revenue']))

pprint(pd.DataFrame(growth_data['annual_assets']))

pprint(pd.DataFrame(growth_data['monthly_revenue']))

pprint(pd.DataFrame(growth_data['monthly_growth']))


2024-10-25 15:04:46,725 - INFO - 데이터 검증 완료


2024-10-25 15:04:46,736 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 15:04:46,743 - INFO - 2023년 데이터 처리 완료
2024-10-25 15:04:46,749 - INFO - 2022년 데이터 처리 완료
2024-10-25 15:04:46,754 - INFO - 2021년 데이터 처리 완료
2024-10-25 15:04:46,760 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 15:04:46,779 - INFO - 최근 12개월 데이터 처리 완료


'2024-08'
                2021년     2022년     2023년  2024년(E)
매출액          61479.00  75081.00  86853.00  62438.00
매출액증가율           1.08     22.12     15.68    -28.11
업종평균 매출액증가율     -2.91      3.30     -3.74      5.25
                2021년     2022년     2023년  2024년(E)
총자산          28837.00  31766.00  30363.00  29464.00
총자산증가율         -10.06     10.16     -4.42     -2.96
업종평균 총자산증가율      6.99      0.72      1.25     -0.24
         2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월매출액       4126     8500     7531     5056     9648     5357     2374   
전년동월매출액     3198     8710     9505     3898     8006     9042     5808   

         2024-04  2024-05  2024-06  2024-07  2024-08  
당월매출액       5943     3978     7491     4997     1837  
전년동월매출액     9022     1895     8730     9864     9273  
          2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
매출액증가율      29.02    -2.41   -20.77    29.71    20.51   -40.75   -59.13   
누적매출액증가율    29.02     6.03    -5.87 

### Profitability (수익성) 전처리

In [68]:
# 재현성을 위한 Random seed 설정
np.random.seed(42)

import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any, Optional
import logging
from pprint import pprint

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

class DataValidationError(Exception):
    """데이터 검증 관련 커스텀 예외"""
    pass

class ProfitabilityMetricsCalculator:
    """수익성 지표 계산을 위한 메인 클래스"""
    
    # 클래스 상수 정의
    REQUIRED_COLUMNS = ['기업명', '날짜', '업종', '영업이익', '당기순이익', '매출액']
    OPERATING_PROFIT_COL = '영업이익'
    NET_PROFIT_COL = '당기순이익'
    REVENUE_COL = '매출액'
    COMPANY_COL = '기업명'
    DATE_COL = '날짜'
    INDUSTRY_COL = '업종'
    
    def __init__(self, df_company_info: pd.DataFrame, target_company_name: str, n_years: int = 3):
        self._validate_input_data(df_company_info, target_company_name)
        self.df = df_company_info[self.REQUIRED_COLUMNS].copy()
        self.target_company_name = target_company_name
        self.n_years = n_years
        self.profitability_data = {}

    def _validate_input_data(self, df: pd.DataFrame, company_name: str) -> None:
        """입력 데이터 유효성 검증"""
        try:
            if not all(col in df.columns for col in self.REQUIRED_COLUMNS):
                missing_cols = set(self.REQUIRED_COLUMNS) - set(df.columns)
                raise DataValidationError(f"필수 컬럼 누락: {missing_cols}")
            
            if company_name not in df[self.COMPANY_COL].unique():
                raise DataValidationError(f"회사를 찾을 수 없음: {company_name}")
            
            if df.empty:
                raise DataValidationError("빈 데이터프레임")
                
            numeric_cols = [self.OPERATING_PROFIT_COL, self.NET_PROFIT_COL, self.REVENUE_COL]
            for col in numeric_cols:
                if not pd.to_numeric(df[col], errors='coerce').notnull().all():
                    raise DataValidationError(f"숫자가 아닌 값이 포함됨: {col}")
                    
            logger.info("데이터 검증 완료")
        except Exception as e:
            logger.error(f"데이터 검증 실패: {str(e)}")
            raise

    def _handle_missing_values(self) -> None:
        """결측치 처리"""
        numeric_cols = [self.OPERATING_PROFIT_COL, self.NET_PROFIT_COL, self.REVENUE_COL]
        for col in numeric_cols:
            missing_count = self.df[col].isnull().sum()
            if missing_count > 0:
                logger.warning(f"{col} 컬럼 결측치 발견: {missing_count}개")
                self.df[col] = self.df[col].fillna(0)

    def _prepare_data(self) -> None:
        """기초 데이터 준비"""
        try:
            self.df['date'] = pd.to_datetime(self.df[self.DATE_COL])
            self.df['year'] = self.df['date'].dt.year
            self.df['month'] = self.df['date'].dt.month
            
            self._handle_missing_values()
            
            self.df_company = self.df[self.df[self.COMPANY_COL] == self.target_company_name].copy()
            self.df_company.reset_index(drop=True, inplace=True)
            
            if self.df_company.empty:
                raise DataValidationError(f"회사 데이터가 없음: {self.target_company_name}")
            
            self.target_industry = self.df_company[self.INDUSTRY_COL].iloc[0]
            self.df_industry = self.df[self.df[self.INDUSTRY_COL] == self.target_industry].copy()
            self.df_industry.reset_index(drop=True, inplace=True)
            
            # 접근 가능한 최신 날짜 설정
            self.latest_date = access_datetime - pd.DateOffset(months=1)
            self.latest_year = self.latest_date.year
            self.latest_month = self.latest_date.month
            
            logger.info(f"데이터 준비 완료 - 기준일자: {self.latest_date.strftime('%Y-%m-%d')}")
        except Exception as e:
            logger.error(f"데이터 준비 실패: {str(e)}")
            raise

    def _calculate_margin_rate(self, profit: float, revenue: float, threshold: float = 0.0001) -> float:
        """이익률 계산"""
        try:
            if abs(revenue) < threshold:
                logger.warning(f"매출액이 임계값보다 작음 (revenue: {revenue})")
                return 0
            margin_rate = round((profit / revenue * 100), 2)
            logger.debug(f"이익률 계산: profit={profit}, revenue={revenue}, rate={margin_rate}")
            return margin_rate
        except Exception as e:
            logger.warning(f"이익률 계산 실패: {str(e)}")
            return 0

    def _process_historical_data(self) -> None:
        """과거 3년 데이터 처리"""
        try:
            for year_offset in range(1, 4):
                target_year = self.latest_year - year_offset
                year_key = f"{target_year}년"
                
                current_year_company = self.df_company[self.df_company['year'] == target_year]
                prev_year_company = self.df_company[self.df_company['year'] == (target_year - 1)]
                
                current_year_industry = self.df_industry[self.df_industry['year'] == target_year]
                prev_year_industry = self.df_industry[self.df_industry['year'] == (target_year - 1)]
                
                company_metrics = self._calculate_metrics(
                    current_year_company, prev_year_company,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=False
                )
                
                industry_metrics = self._calculate_metrics(
                    current_year_industry, prev_year_industry,
                    f"{target_year}-12", f"{target_year-1}-12",
                    is_industry=True
                )
                
                self._update_profitability_data(year_key, company_metrics, industry_metrics)
                logger.info(f"{year_key} 데이터 처리 완료")
        except Exception as e:
            logger.error(f"과거 데이터 처리 실패: {str(e)}")
            raise

    def _calculate_metrics(self, current_data: pd.DataFrame, prev_data: pd.DataFrame,
                        current_period: str, prev_period: str, is_industry: bool = False) -> Dict[str, float]:
        """수익성 지표 계산"""
        try:
            if current_data.empty:
                logger.warning(f"현재 데이터 없음: {current_period}")
                return {
                    'operating_profit': 0,
                    'net_profit': 0,
                    'operating_profit_margin': 0,
                    'net_profit_margin': 0
                }
            
            if is_industry:
                # 업종 평균 계산
                current_op = current_data[self.OPERATING_PROFIT_COL].mean()
                current_np = current_data[self.NET_PROFIT_COL].mean()
                current_rev = current_data[self.REVENUE_COL].mean()
            else:
                # 개별 기업 계산
                current_op = current_data[self.OPERATING_PROFIT_COL].sum()
                current_np = current_data[self.NET_PROFIT_COL].sum()
                current_rev = current_data[self.REVENUE_COL].sum()
            
            metrics = {
                'operating_profit': int(round(current_op)),
                'net_profit': int(round(current_np)),
                'operating_profit_margin': round((current_op / current_rev * 100), 2) if current_rev != 0 else 0,
                'net_profit_margin': round((current_np / current_rev * 100), 2) if current_rev != 0 else 0
            }
            
            logger.debug(f"지표 계산 완료: {metrics}")
            return metrics
        except Exception as e:
            logger.error(f"지표 계산 실패: {str(e)}")
            return {
                'operating_profit': 0,
                'net_profit': 0,
                'operating_profit_margin': 0,
                'net_profit_margin': 0
            }



    def _process_estimate_data(self) -> None:
        """연말 예상 데이터 처리"""
        try:
            estimate_key = f"{self.latest_year}년(E)"
            
            current_mask = (self.df_company['year'] == self.latest_year) & \
                          (self.df_company['month'] <= self.latest_month)
            current_company = self.df_company[current_mask].copy()
            current_company.reset_index(drop=True, inplace=True)
            
            industry_mask = (self.df_industry['year'] == self.latest_year) & \
                          (self.df_industry['month'] <= self.latest_month)
            current_industry = self.df_industry[industry_mask].copy()
            current_industry.reset_index(drop=True, inplace=True)
            
            company_metrics = self._calculate_annualized_metrics(current_company, is_industry=False)
            industry_metrics = self._calculate_annualized_metrics(current_industry, is_industry=True)
            
            self._update_profitability_data(estimate_key, company_metrics, industry_metrics)
            logger.info("연말 예상 데이터 처리 완료")
        except Exception as e:
            logger.error(f"연말 예상 데이터 처리 실패: {str(e)}")
            raise

    def _process_recent_data(self) -> None:
        """최근 12개월 데이터 처리"""
        try:
            end_date = self.latest_date
            start_date = end_date - pd.DateOffset(months=11)
            
            past_12_months = pd.date_range(start=start_date, end=end_date, freq='MS')
            cumulative_data = {
                'current': {
                    'operating_profit': 0,
                    'net_profit': 0,
                    'revenue': 0
                },
                'previous': {
                    'operating_profit': 0,
                    'net_profit': 0,
                    'revenue': 0
                }
            }

            for idx, date in enumerate(past_12_months):
                year = date.year
                month = date.month

                current_month = self.df_company[
                    (self.df_company['year'] == year) & 
                    (self.df_company['month'] == month)
                ]

                prev_year = year - 1
                prev_month = self.df_company[
                    (self.df_company['year'] == prev_year) & 
                    (self.df_company['month'] == month)
                ]

                month_str = f"{year}-{month:02d}"
                self._update_monthly_data(month_str, current_month, prev_month,
                                        cumulative_data, idx == 0)

            logger.info("최근 12개월 데이터 처리 완료")
        except Exception as e:
            logger.error(f"최근 데이터 처리 실패: {str(e)}")
            raise

    def _calculate_annualized_metrics(self, current_data: pd.DataFrame, is_industry: bool = False) -> Dict[str, float]:
        """연환산 지표 계산"""
        try:
            if current_data.empty:
                logger.warning("현재 데이터 없음")
                return {
                    'operating_profit': 0,
                    'net_profit': 0,
                    'operating_profit_margin': 0,
                    'net_profit_margin': 0
                }
            
            if is_industry:
                # 업종 평균 계산
                monthly_op = current_data[self.OPERATING_PROFIT_COL].mean()
                monthly_np = current_data[self.NET_PROFIT_COL].mean()
                monthly_rev = current_data[self.REVENUE_COL].mean()
            else:
                # 개별 기업 계산
                monthly_op = current_data[self.OPERATING_PROFIT_COL].sum()
                monthly_np = current_data[self.NET_PROFIT_COL].sum()
                monthly_rev = current_data[self.REVENUE_COL].sum()
            
            # 연환산 계산 (단순 곱하기 방식)
            annualized_op = (monthly_op / self.latest_month) * 12
            annualized_np = (monthly_np / self.latest_month) * 12
            annualized_rev = (monthly_rev / self.latest_month) * 12
            
            metrics = {
                'operating_profit': int(round(annualized_op)),
                'net_profit': int(round(annualized_np)),
                'operating_profit_margin': round((annualized_op / annualized_rev * 100), 2) if annualized_rev != 0 else 0,
                'net_profit_margin': round((annualized_np / annualized_rev * 100), 2) if annualized_rev != 0 else 0
            }
            
            logger.debug(f"연환산 지표 계산 완료: {metrics}")
            return metrics
        except Exception as e:
            logger.error(f"연환산 지표 계산 실패: {str(e)}")
            return {
                'operating_profit': 0,
                'net_profit': 0,
                'operating_profit_margin': 0,
                'net_profit_margin': 0
            }


    def _update_monthly_data(self, month: str, current_data: pd.DataFrame,
                           prev_data: pd.DataFrame, cumulative_data: Dict[str, Dict[str, float]],
                           is_first_month: bool) -> None:
        """월별 데이터 업데이트"""
        try:
            if current_data.empty:
                self._set_default_monthly_values(month)
                return

            # 영업이익 관련 데이터
            current_op = current_data[self.OPERATING_PROFIT_COL].iloc[0]
            prev_op = prev_data[self.OPERATING_PROFIT_COL].iloc[0] if not prev_data.empty else 0
            
            # 당기순이익 관련 데이터
            current_np = current_data[self.NET_PROFIT_COL].iloc[0]
            # print(current_np)
            prev_np = prev_data[self.NET_PROFIT_COL].iloc[0] if not prev_data.empty else 0
            
            # 매출액 관련 데이터
            current_rev = current_data[self.REVENUE_COL].iloc[0]
            prev_rev = prev_data[self.REVENUE_COL].iloc[0] if not prev_data.empty else 0
            
            # 누적 데이터 업데이트
            cumulative_data['current']['operating_profit'] += current_op
            cumulative_data['current']['net_profit'] += current_np
            cumulative_data['current']['revenue'] += current_rev
            
            if not prev_data.empty:
                cumulative_data['previous']['operating_profit'] += prev_op
                cumulative_data['previous']['net_profit'] += prev_np
                cumulative_data['previous']['revenue'] += prev_rev

            # 증가율 계산
            op_margin = self._calculate_margin_rate(current_op, current_rev)
            np_margin = self._calculate_margin_rate(current_np, current_rev)
            
            # 누적 증가율 계산
            cumulative_op_margin = self._calculate_margin_rate(
                cumulative_data['current']['operating_profit'] ,
                cumulative_data['current']['revenue']
            )
            cumulative_np_margin = self._calculate_margin_rate(
                cumulative_data['current']['net_profit'],
                cumulative_data['current']['revenue']
            )

            if is_first_month:
                cumulative_op_margin = op_margin
                cumulative_np_margin = np_margin
            
            self.profitability_data['recent_data'][month] = {
                'operating_profit': int(round(current_op)),
                'prev_operating_profit': int(round(prev_op)),
                'operating_profit_margin': op_margin,
                'cumulative_op_margin': cumulative_op_margin,
                'net_profit': int(round(current_np)),
                'prev_net_profit': int(round(prev_np)),
                'net_profit_margin': np_margin,
                'cumulative_np_margin': cumulative_np_margin
            }
            logger.debug(f"{month} 월별 데이터 업데이트 완료")
        except Exception as e:
            logger.error(f"월별 데이터 업데이트 실패: {str(e)}")
            self._set_default_monthly_values(month)

    def _set_default_monthly_values(self, month: str) -> None:
        """월별 기본값 설정"""
        try:
            self.profitability_data['recent_data'][month] = {
                'operating_profit': 0,
                'prev_operating_profit': 0,
                'operating_profit_margin': 0,
                'cumulative_op_margin': 0,
                'net_profit': 0,
                'prev_net_profit': 0,
                'net_profit_margin': 0,
                'cumulative_np_margin': 0    
            }
            logger.debug(f"{month} 월별 기본값 설정 완료")
        except Exception as e:
            logger.error(f"월별 기본값 설정 실패: {str(e)}")

    def _get_previous_year_data(self, current_year: int, is_industry: bool = False) -> pd.DataFrame:
        """전년도 데이터 조회"""
        try:
            if is_industry:
                prev_year_data = self.df_industry[self.df_industry['year'] == (current_year - 1)].copy()
            else:
                prev_year_data = self.df_company[self.df_company['year'] == (current_year - 1)].copy()
            
            prev_year_data.reset_index(drop=True, inplace=True)
            return prev_year_data
        except Exception as e:
            logger.error(f"전년도 데이터 조회 실패: {str(e)}")
            return pd.DataFrame()

    def _update_profitability_data(self, key: str, company_metrics: Dict[str, float],
                                industry_metrics: Dict[str, float]) -> None:
        """수익성 데이터 업데이트"""
        try:
            self.profitability_data['year_level_data'][key] = {
                'operating_profit': company_metrics['operating_profit'],
                'net_profit': company_metrics['net_profit']
            }
            self.profitability_data['year_rate_data'][key] = {
                'operating_profit_margin': company_metrics['operating_profit_margin'],
                'net_profit_margin': company_metrics['net_profit_margin'],
                'industry_operating_profit_margin': industry_metrics['operating_profit_margin'],
                'industry_net_profit_margin': industry_metrics['net_profit_margin']
            }
            logger.debug(f"{key} 수익성 데이터 업데이트 완료")
        except Exception as e:
            logger.error(f"수익성 데이터 업데이트 실패: {str(e)}")
            raise
        
    def calculate_profitability_metrics(self) -> Dict[str, pd.DataFrame]:
        """수익성 지표 계산 실행"""
        try:
            self._prepare_data()
            
            self.profitability_data = {
                'latest_year_month': self.latest_date.strftime('%Y-%m'),
                'year_level_data': {},
                'year_rate_data': {},
                'recent_data': {}
            }
            
            self._process_historical_data()
            self._process_estimate_data()
            self._process_recent_data()
            
            # 연간 데이터 재구성
            years = sorted(self.profitability_data['year_level_data'].keys())
            
            annual_profit = {
                '영업이익': [self.profitability_data['year_level_data'][year]['operating_profit'] for year in years],
                '영업이익률': [self.profitability_data['year_rate_data'][year]['operating_profit_margin'] for year in years],
                '업종평균 영업이익률' : [self.profitability_data['year_rate_data'][year]['industry_operating_profit_margin'] for year in years] 
            }
            df_annual_profit = pd.DataFrame(annual_profit, index=years).T
            
            annual_margins = {
                '당기순이익': [self.profitability_data['year_level_data'][year]['net_profit'] for year in years],
                '당기순이익률': [self.profitability_data['year_rate_data'][year]['net_profit_margin'] for year in years],
                '업종평균 당기순이익률' : [self.profitability_data['year_rate_data'][year]['industry_net_profit_margin'] for year in years]
            }
            df_annual_margins = pd.DataFrame(annual_margins, index=years).T
            
            # 월별 데이터 재구성
            months = sorted(self.profitability_data['recent_data'].keys())
            
            monthly_profit = {
                '당월영업이익': [self.profitability_data['recent_data'][month]['operating_profit'] for month in months],
                '전년동월영업이익': [self.profitability_data['recent_data'][month]['prev_operating_profit'] for month in months],
                '당월당기순이익': [self.profitability_data['recent_data'][month]['net_profit'] for month in months],
                '전년동월당기순이익': [self.profitability_data['recent_data'][month]['prev_net_profit'] for month in months],
                
            }
            df_monthly_profit = pd.DataFrame(monthly_profit, index=months).T
            
            monthly_margins = {
            '영업이익률': [self.profitability_data['recent_data'][month]['operating_profit_margin'] for month in months],
            '누적영업이익률': [self.profitability_data['recent_data'][month]['cumulative_op_margin'] for month in months],
            '당기순이익률': [self.profitability_data['recent_data'][month]['net_profit_margin'] for month in months],
            '누적순이익률': [self.profitability_data['recent_data'][month]['cumulative_np_margin'] for month in months]
            }
            df_monthly_margins = pd.DataFrame(monthly_margins, index=months).T
            
            return {
                'latest_year_month': self.latest_date.strftime('%Y-%m'),
                'annual_profit': df_annual_profit,
                'annual_margins': df_annual_margins,
                'monthly_profit': df_monthly_profit,
                'monthly_margins': df_monthly_margins
            }
            
        except Exception as e:
            logger.error(f"수익성 지표 계산 실패: {str(e)}")
            raise

def preprocess_profitability_data(df_company_info: pd.DataFrame, target_company_name: str) -> Dict[str, Any]:
    """
    수익성 지표 데이터를 전처리하여 JSON 형식으로 변환
    
    Args:
        df_company_info: 전체 회사 재무 데이터
        target_company_name: 대상 회사명
        
    Returns:
        Dict[str, Any]: 전처리된 수익성 지표 데이터
    """
    try:
        calculator = ProfitabilityMetricsCalculator(df_company_info, target_company_name)
        return calculator.calculate_profitability_metrics()
    except Exception as e:
        logger.error(f"전처리 실패: {str(e)}")
        raise

profitability_data = preprocess_profitability_data(df_company_info, 'Company_1')

# profitability_data.keys()
pprint(pd.DataFrame(profitability_data['annual_profit']))

pprint(pd.DataFrame(profitability_data['annual_margins']))

pprint(pd.DataFrame(profitability_data['monthly_profit']))

pprint(pd.DataFrame(profitability_data['monthly_margins']))

2024-10-25 15:05:21,095 - INFO - 데이터 검증 완료
2024-10-25 15:05:21,105 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01


2024-10-25 15:05:21,109 - INFO - 2023년 데이터 처리 완료
2024-10-25 15:05:21,114 - INFO - 2022년 데이터 처리 완료
2024-10-25 15:05:21,116 - INFO - 2021년 데이터 처리 완료
2024-10-25 15:05:21,120 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 15:05:21,137 - INFO - 최근 12개월 데이터 처리 완료


              2021년    2022년    2023년  2024년(E)
영업이익        1312.00 -3259.00  1977.00   1666.00
영업이익률          2.13    -4.34     2.28      2.67
업종평균 영업이익률    -0.22    -1.37     0.63      0.16
              2021년  2022년    2023년  2024년(E)
당기순이익        217.00 -84.00  1150.00    260.00
당기순이익률         0.35  -0.11     1.32      0.42
업종평균 당기순이익률   -0.28   0.22    -0.27     -0.14
           2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월영업이익         -28     -517    -1281     -341      830      -68     -313   
전년동월영업이익      -211      326      557       98      138     1587     -462   
당월당기순이익        -13      235      -30     -254      394      -23      155   
전년동월당기순이익     -131       57     -122      -14       80      817       60   

           2024-04  2024-05  2024-06  2024-07  2024-08  
당월영업이익         581     -383      465      -90       89  
전년동월영업이익      -182      -91     1382     1622      150  
당월당기순이익         18     -179     -133      -46      -13  
전년동월당기순이익      

## 4. Langchain을 활용한 분석

- cache 설정 : 비용 절감 목적
- streaming을 위한 custom handler 설정
- common_llm : 데이터 기반 해석만 하는 자유도가 낮은 gpt-4o-mini
- load_template : template 읽기
- determine_strength_weakness : **BPI 계산법 통해 연계 필요**
- create_analysis_chain : 분석 chain 생성
- run_analysis : 강/약점을 활용해 분석 시작

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any
import logging
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from operator import itemgetter

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# SQLite 캐시 설정 (주석 처리됨)
# try:
#     langchain_cache = SQLiteCache(database_path="cache.sqlite")
# except Exception as e:
#     print(f"Error initializing cache: {e}") 
#     langchain_cache = None

# OpenAI GPT-4 Mini 모델 설정
common_llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=300,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 디렉토리 설정
CURRENT_DIR = os.getcwd()
PROMPT_PATH = os.path.join(CURRENT_DIR, "prompts")

def load_prompt(file_name: str) -> str:
    file_path = os.path.join(PROMPT_PATH, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: Prompt file {file_name} not found.")
        return ""

def determine_strength_weakness(data: pd.DataFrame) -> Dict[str, str]:
    growth_data = preprocess_growth_data(data, data['기업명'].iloc[0])
    profitability_data = preprocess_profitability_data(data, data['기업명'].iloc[0])
    
    print(growth_data['annual_revenue'])
    
    # 성장성과 수익성 점수 비교
    growth_score = float(growth_data['monthly_growth'].loc['누적매출액증가율'].iloc[-1])
    profit_score = float(profitability_data['monthly_margins'].loc['누적영업이익률'].iloc[-1])
    
    # 디버깅을 위한 출력
    print(f"Growth Score: {growth_score}, Profit Score: {profit_score}")
    
    if growth_score > profit_score:
        return {'strength': 'growth', 'weakness': 'profitability'}
    else:
        return {'strength': 'profitability', 'weakness': 'growth'}

def create_analysis_chain(indicator: str, is_strength: bool, llm: ChatOpenAI):
    base_template = load_prompt(f"{indicator}_template.txt")
    additional_prompt = "Perform analysis on positive side." if is_strength else "Perform analysis on negative side."
    full_template = f"{base_template}\n\n{additional_prompt}"
    
    prompt = PromptTemplate.from_template(full_template)
    analysis_chain = prompt | llm | StrOutputParser()
    
    if indicator == 'growth':
        preprocess_func = preprocess_growth_data
        data_keys = ['latest_year_month', 'annual_revenue', 'annual_assets', 'monthly_revenue', 'monthly_growth']
    elif indicator == 'profitability':
        preprocess_func = preprocess_profitability_data
        data_keys = ['latest_year_month', 'annual_profit', 'annual_margins', 'monthly_profit', 'monthly_margins']
    else:
        raise ValueError(f"Unknown indicator: {indicator}")
        
    return RunnableLambda(lambda df: {
        **{key: preprocess_func(df, df['기업명'].iloc[0])[key] for key in data_keys}
    })

def run_analysis(df: pd.DataFrame) -> Dict[str, Any]:
    strength_weakness = determine_strength_weakness(df)
    strength = strength_weakness['strength']
    weakness = strength_weakness['weakness']
    
    strength_chain = create_analysis_chain(strength, True, common_llm)
    weakness_chain = create_analysis_chain(weakness, False, common_llm)
    
    # 병렬로 강점과 약점 분석을 수행하는 체인 생성
    parallel_chain = RunnableParallel({
        'strength_analysis': strength_chain,
        'weakness_analysis': weakness_chain
    })
    
    return {
        'parallel': parallel_chain,
        'strength': strength_chain,
        'weakness': weakness_chain,
        'strength_name': strength,
        'weakness_name': weakness
    }

def merge_analysis_results(strength_result: Any, weakness_result: Any) -> str:
    # 결과가 dict일 경우 문자열로 변환
    if isinstance(strength_result, dict):
        strength_result = pprint(strength_result, indent=2)  # dict를 보기 좋게 출력
    if isinstance(weakness_result, dict):
        weakness_result = pprint(weakness_result, indent=2)
    
    return '\n\n'.join([
        "Strength Analysis:",
        str(strength_result),  # dict 형식을 문자열로 변환
        "Weakness Analysis:",
        str(weakness_result)
    ])


def print_preprocessed_data(df: pd.DataFrame, metric: str) -> None:
    preprocess_func = globals()[f"preprocess_{metric}_data"]
    preprocessed_data = preprocess_func(df, df['기업명'].iloc[0])
    print(f"\n{metric.capitalize()} Data:")
    for key, value in preprocessed_data.items():
        if isinstance(value, pd.DataFrame):
            print(f"\n{key}:")
            print(value)

NameError: name 'StreamingStdOutCallbackHandler' is not defined

## 5. 경영제언 Session
- gpt-4o로 고품질 모델 활용
- 창의도를 결정하는 temperature=1.2로 설정
- max_tokens=500으로 조금 더 긴 text 작성하도록 설정
- final_chain : 앞에서 분석한 strength와 weakness를 합쳐서 경영제언을 하도록 chain으로 연결
    - 1안 : 앞에서 streaming으로 분석한 strength_result와 weakness_result를 묶어서 수행 (토큰 비용 아낌)
    - 2안 : End-to-End로 강단점 분석부터 시작해서 경영 제언까지 수행 (토큰 비용 더 발생) 

In [110]:
insight_llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=1.2,
    max_tokens=500,
    streaming=True,
    # cache=langchain_cache,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [113]:
# 분석 실행
company_list = df_company_info['기업명'].unique()[:3]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]
    chains = run_analysis(firm_data)
    
    strength_chain = chains['strength']
    weakness_chain = chains['weakness']
    strength_name = chains['strength_name']
    weakness_name = chains['weakness_name']
    
    print(f"\n\n{'#'*10} Preprocessing Strength ({strength_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(firm_data, strength_name)
    
    print(f"\n\n{'#'*10} Analyzing Strength ({strength_name}) for {firm} {'#'*10}")
    strength_result = strength_chain.invoke(firm_data)
    print(strength_result)
    
    print(f"\n\n{'#'*10} Preprocessing Weakness ({weakness_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(firm_data, weakness_name)
        
    print(f"\n\n{'#'*10} Analyzing Weakness ({weakness_name}) for {firm} {'#'*10}")
    weakness_result = weakness_chain.invoke(firm_data)
    print(weakness_result)
    
    print("\n\n",'#'*10, f'Final Comment for {firm}', '#'*10)    
    final_chain = (
        RunnableParallel({
            'strength_result': itemgetter('strength'),
            'weakness_result': itemgetter('weakness')
        })
        | RunnableLambda(lambda x: {'info': merge_analysis_results(x['strength_result'], x['weakness_result'])})
        | PromptTemplate.from_template(
        """
        Translate and analyze the provided company's strengths and weaknesses:

        {info}

        - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
        - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
        - Highlight critical trends and figures as evidence to support your recommendations.
        - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

        # Output Format
        - The response should be in Korean, using natural and connected language.
        - Limit the response to 5 lines or less, with a clear and cohesive structure.
        - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

        # Guidelines
        1. Base your response objectively on the given data without speculation.
        2. Use specific figures and trends to support your analysis.
        3. Provide practical recommendations that are directly actionable.
        4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
        """
        ) 
        | insight_llm 
    )
    result = final_chain.invoke({
        'strength': strength_result,
        'weakness': weakness_result
    })
    
    print(result)


2024-10-25 17:54:47,442 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,445 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,449 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,454 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,458 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,463 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 17:54:47,483 - INFO - 최근 12개월 데이터 처리 완료
2024-10-25 17:54:47,487 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,493 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,496 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,500 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,503 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,507 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 17:54:47,527 - INFO - 최근 12개월 데이터 처리 완료
2024-10-25 17:54:47,534 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,539 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,543 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,546 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,549 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,552 - INFO - 연말 예상 데이터 처

                2021년     2022년     2023년  2024년(E)
매출액          61479.00  75081.00  86853.00  62438.00
매출액증가율           1.08     22.12     15.68    -28.11
업종평균 매출액증가율      1.08     22.12     15.68    -28.11
Growth Score: -23.13, Profit Score: -1.58


########## Preprocessing Strength (profitability) Data for Company_1 ##########

Profitability Data:

annual_profit:
              2021년    2022년    2023년  2024년(E)
영업이익        1312.00 -3259.00  1977.00   1666.00
영업이익률          2.13    -4.34     2.28      2.67
업종평균 영업이익률     2.13    -4.34     2.28      2.67

annual_margins:
              2021년  2022년    2023년  2024년(E)
당기순이익        217.00 -84.00  1150.00    260.00
당기순이익률         0.35  -0.11     1.32      0.42
업종평균 당기순이익률    0.35  -0.11     1.32      0.42

monthly_profit:
           2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월영업이익         -28     -517    -1281     -341      830      -68     -313   
전년동월영업이익      -211      326      557       98      138     1587     -

2024-10-25 17:54:47,736 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,740 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,743 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,747 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,750 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,753 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 17:54:47,766 - INFO - 최근 12개월 데이터 처리 완료
2024-10-25 17:54:47,773 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,779 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,782 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,785 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,787 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,789 - INFO - 연말 예상 데이터 처리 완료
2024-10-25 17:54:47,799 - INFO - 최근 12개월 데이터 처리 완료
2024-10-25 17:54:47,802 - INFO - 데이터 검증 완료
2024-10-25 17:54:47,806 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-10-25 17:54:47,808 - INFO - 2023년 데이터 처리 완료
2024-10-25 17:54:47,813 - INFO - 2022년 데이터 처리 완료
2024-10-25 17:54:47,817 - INFO - 2021년 데이터 처리 완료
2024-10-25 17:54:47,821 - INFO - 연말 예상 데이터 처

{'latest_year_month': '2024-08', 'annual_profit':               2021년    2022년    2023년  2024년(E)
영업이익        1312.00 -3259.00  1977.00   1666.00
영업이익률          2.13    -4.34     2.28      2.67
업종평균 영업이익률     2.13    -4.34     2.28      2.67, 'annual_margins':               2021년  2022년    2023년  2024년(E)
당기순이익        217.00 -84.00  1150.00    260.00
당기순이익률         0.35  -0.11     1.32      0.42
업종평균 당기순이익률    0.35  -0.11     1.32      0.42, 'monthly_profit':            2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월영업이익         -28     -517    -1281     -341      830      -68     -313   
전년동월영업이익      -211      326      557       98      138     1587     -462   
당월당기순이익        -13      235      -30     -254      394      -23      155   
전년동월당기순이익     -131       57     -122      -14       80      817       60   

           2024-04  2024-05  2024-06  2024-07  2024-08  
당월영업이익         581     -383      465      -90       89  
전년동월영업이익      -182      -91     1382     

2024-10-25 17:54:48,619 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

{'latest_year_month': '2024-08', 'annual_revenue':                 2021년     2022년     2023년  2024년(E)
매출액          75716.00  57006.00  56611.00  68559.00
매출액증가율          22.76    -24.71     -0.69     21.11
업종평균 매출액증가율     22.76    -24.71     -0.69     21.11, 'annual_assets':                 2021년    2022년     2023년  2024년(E)
총자산          31975.00  29034.0  32027.00  29408.00
총자산증가율           2.68     -9.2     10.31     -8.18
업종평균 총자산증가율      2.68     -9.2     10.31     -8.18, 'monthly_revenue':          2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월매출액       3179     5401     1705     6902     1681     9907     7636   
전년동월매출액     1293     6857     2279     4798     4685     3653     2613   

         2024-04  2024-05  2024-06  2024-07  2024-08  
당월매출액       7706     1523     5750     4403     7100  
전년동월매출액     8380     4291     3629     5670     6503  , 'monthly_growth':           2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
매출액증가율     145.86

2024-10-25 17:54:53,656 - INFO - 최근 12개월 데이터 처리 완료


{'latest_year_month': '2024-08', 'annual_profit':              2021년   2022년   2023년  2024년(E)
영업이익       -472.00  1195.0  603.00    -963.0
영업이익률        -0.62     2.1    1.07      -1.4
업종평균 영업이익률   -0.62     2.1    1.07      -1.4, 'annual_margins':                2021년   2022년   2023년  2024년(E)
당기순이익        1387.00  296.00 -572.00  -2097.00
당기순이익률          1.83    0.52   -1.01     -3.06
업종평균 당기순이익률     1.83    0.52   -1.01     -3.06, 'monthly_profit':            2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월영업이익        -633      662     -161     -478      207     1293    -1518   
전년동월영업이익       217      345       12      567      248      590     -275   
당월당기순이익        -44     -287      -98      210       57      315    -1110   
전년동월당기순이익      146     -132        2      148      -91       21      -15   

           2024-04  2024-05  2024-06  2024-07  2024-08  
당월영업이익        -467     -262      488     -857      474  
전년동월영업이익      -717      499     -474      481    

2024-10-25 17:54:54,178 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


제공된 데이터에 회사의 강점과 약점이 명시되지 않았으므로, 이에 대한 구체적인 분석은 진행할 수 없습니다. 따라서 현재 주어진 정보에 기반한 객관적인 행동을 권장합니다. 첫째, 회사의 잠재적인 강점을 규명하기 위한 세심한 내부 평가가 필요합니다. 둘째, 데이터 누락의 위험요소를 완화하기 위해 데이터 관리 및 수집 체계의 개선이 필수적입니다. 마지막으로 지속 가능한 경쟁 우위를 위해 소비자 의견을 광범위하게 수렴해야 합니다.content='제공된 데이터에 회사의 강점과 약점이 명시되지 않았으므로, 이에 대한 구체적인 분석은 진행할 수 없습니다. 따라서 현재 주어진 정보에 기반한 객관적인 행동을 권장합니다. 첫째, 회사의 잠재적인 강점을 규명하기 위한 세심한 내부 평가가 필요합니다. 둘째, 데이터 누락의 위험요소를 완화하기 위해 데이터 관리 및 수집 체계의 개선이 필수적입니다. 마지막으로 지속 가능한 경쟁 우위를 위해 소비자 의견을 광범위하게 수렴해야 합니다.' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90354628f2'} id='run-e5400ae2-b05d-4976-9311-7d28e910daac-0'


In [83]:
profitability_data.keys()

dict_keys(['latest_year_month', 'annual_profit', 'annual_margins', 'monthly_profit', 'monthly_margins'])

In [ ]:


print(tabulate(pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data']), headers = 'keys', tablefmt='psql'))

# pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data'])

In [ ]:
# run_analysis 함수의 결과를 사용하여 final_chain 구성
def create_final_chain(df: pd.DataFrame):
    chains = run_analysis(df)
    parallel_chain = chains['parallel']

    final_chain = (
        parallel_chain
        | RunnableLambda(lambda x: {
            'info': merge_analysis_results(
                x['strength_analysis'],
                x['weakness_analysis']
            ),
            'strength_name': chains['strength_name'],
            'weakness_name': chains['weakness_name']
        })
        | PromptTemplate.from_template(
            """
            Translate and analyze the provided company's strengths ({strength_name}) and weaknesses ({weakness_name}):

            {info}

            - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
            - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
            - Highlight critical trends and figures as evidence to support your recommendations.
            - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

            # Output Format
            - The response should be in Korean.
            - Limit the response to 5 lines or less, with a clear and cohesive structure.
            - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

            # Guidelines
            1. Base your response objectively on the given data without speculation.
            2. Use specific figures and trends to support your analysis.
            3. Provide practical recommendations that are directly actionable.
            4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
            """
        ) 
        | insight_llm 
        | StrOutputParser()
    )

    return final_chain

# 실행

# 단일 회사 정보만을 필터링하여 사용 (예: 첫 번째 회사)
company_list = df_company_info['기업명'].unique()[:3]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]
    final_chain = create_final_chain(firm_data)
    print("\n", '#'*10, f'Final Comment for {firm}', '#'*10)
    result = final_chain.invoke(firm_data)
    print('\n')


In [ ]:
print(result)